In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # visualization
import seaborn as sns # visualization
import cv2
import tensorflow as tf 
from tensorflow import keras
from pathlib import Path
import PIL
import os


In [ ]:
glaucoma=Path("dataset/glaucoma")
cataract=Path("dataset/cataract")
normal=Path("dataset/normal")
diabetic_retinopathy=Path("dataset/diabetic_retinopathy")

In [ ]:
disease_type = [glaucoma, cataract,normal,diabetic_retinopathy]
df = pd.DataFrame()
from tqdm import tqdm
for types in disease_type:
    for imagepath in tqdm(list(types.iterdir()), desc= str(types)):
        df = pd.concat([df, pd.DataFrame({'image': [str(imagepath)],'disease_type': [disease_type.index(types)]})], ignore_index=True)


In [ ]:
df

In [ ]:
df.disease_type.value_counts()

In [ ]:
def plot_image(n, num_samples=3):
    disease_labels = ['glaucoma', 'cataract', 'normal', 'diabetic_retinopathy']
    images = df[df['disease_type'] == n].sample(num_samples)['image']

    plt.figure(figsize=(12, 12))

    for i, path in enumerate(images, 1):
        img = (plt.imread(path) - plt.imread(path).min()) / plt.imread(path).max()
        plt.subplot(3, 3, i)
        plt.imshow(img)
        plt.axis('off')
        plt.title(disease_labels[n])

    plt.show()

In [ ]:
plot_image(1)

In [ ]:
plot_image(0)

In [ ]:
plot_image(2)

In [ ]:
plot_image(3)

In [ ]:
df['disease_type'] = df['disease_type'].map({0:'glaucoma',1:'cataract',2:'normal',3:'diabetic_retinopathy'})

In [ ]:
df.disease_type.value_counts()

In [ ]:
df1 = df.sample(frac=1).reset_index(drop=True)

In [ ]:
sns.countplot(x = 'disease_type', data = df1)
plt.xlabel("Disease type")
plt.show()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.applications.resnet import ResNet152
from keras.applications.resnet import preprocess_input
from tensorflow import keras
from keras import layers
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.2)

In [ ]:
train_data = datagen.flow_from_dataframe(dataframe=df1,
                                          x_col ='image',
                                          y_col = 'disease_type',
                                          target_size=(224,224),
                                          class_mode = 'categorical',
                                          batch_size = 32,
                                          shuffle = True,
                                          subset = 'training')


In [ ]:
valid_data = datagen.flow_from_dataframe(dataframe=df1,
                                          x_col ='image',
                                          y_col = 'disease_type',
                                          target_size=(224,224),
                                          class_mode = 'categorical',
                                          batch_size = 32,
                                          shuffle = False,
                                          subset = 'validation')


In [ ]:
labels=[key for key in train_data.class_indices]
num_classes = len(disease_type)

In [ ]:

pretrained_model = tf.keras.applications.ResNet152(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)
pretrained_model.trainable = False

inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer='adam', 
              loss=tf.keras.losses.categorical_crossentropy, 
              metrics=['accuracy']) 
model.summary()

In [ ]:
his = model.fit( 
  train_data,
    validation_data=valid_data, 
  epochs=15 
)
model.save("my_model.h5")

In [ ]:
plt.plot(his.history['accuracy'])
plt.plot(his.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Val'])
plt.show()

In [ ]:
plt.plot(his.history['loss'])
plt.plot(his.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Val'])
plt.show()


In [ ]:
y_test = valid_data.classes
y_pred = model.predict(valid_data)
y_pred = np.argmax(y_pred,axis=1)


In [ ]:
print(classification_report(y_test,y_pred,target_names = labels))


In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred, normalize='true')
plt.figure(figsize = (15, 10))
sns.heatmap(cf_matrix, annot=True, xticklabels = sorted(set(y_test)), yticklabels = sorted(set(y_test)))
plt.title('Normalized Confusion Matrix')
plt.show()

In [ ]:
import keras.utils as image
from keras.applications.resnet import preprocess_input
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("my_model.h5")

# Define the disease labels
labels = ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']

def predict_disease(img_path):
    # Load and preprocess the input image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Get model predictions
    predictions = model.predict(img_array)

    # Get the predicted class index
    predicted_class_index = np.argmax(predictions)

    # Get the predicted disease label
    predicted_disease = labels[predicted_class_index]

    # Display the input image
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Predicted Disease: {predicted_disease}')
    plt.show()

    return predicted_disease

# Example usage:
img_path_to_test = "test/n9.jpg"
predicted_disease = predict_disease(img_path_to_test)
print(f'The predicted disease for the input image is: {predicted_disease}')